# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235414409175                   -0.50    8.0         
  2   -7.250284763576       -1.83       -1.40    1.0   5.15ms
  3   -7.251127389681       -3.07       -2.07    1.0   5.15ms
  4   -7.251186418911       -4.23       -2.03    2.0   6.39ms
  5   -7.251329896738       -3.84       -2.63    1.0   5.22ms
  6   -7.251337973313       -5.09       -3.12    1.0   5.32ms
  7   -7.251338709813       -6.13       -3.73    1.0   5.60ms
  8   -7.251338795792       -7.07       -4.13    2.0   6.63ms
  9   -7.251338798133       -8.63       -4.85    1.0   5.53ms
 10   -7.251338798685       -9.26       -5.27    3.0   7.45ms
 11   -7.251338798701      -10.80       -5.74    1.0   5.67ms
 12   -7.251338798703      -11.74       -5.91    2.0   6.68ms
 13   -7.251338798704      -11.83       -6.53    1.0   5.75ms
 14   -7.251338798705      -13.05       -6.83    2.0   6.78ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07383263154415157
[ Info: Arnoldi iteration step 2: normres = 0.6222518012864504
[ Info: Arnoldi iteration step 3: normres = 0.7518371544180092
[ Info: Arnoldi iteration step 4: normres = 0.2216870716952142
[ Info: Arnoldi iteration step 5: normres = 0.591454011691182
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.04e-02, 8.37e-02, 5.19e-01, 2.69e-01, 1.43e-02)
[ Info: Arnoldi iteration step 6: normres = 0.25459341321689466
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.31e-03, 1.38e-01, 1.62e-01, 9.34e-02, 3.87e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09923096620018791
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.12e-04, 1.37e-02, 1.33e-02, 7.08e-02, 5.99e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11840249136390532
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.07e-0